# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
# create file path
infile = os.path.join("Output", "city_weather_data.csv")

#read csv into a dataframe
city_data = pd.read_csv(infile)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
gmaps.configure(api_key=g_key)

# Create a variable for lat, long
city_locations = city_data[["Lat", "Lng"]]

# Create a variable for humidity'
city_humidity = city_data["Humidity"]

# Create the  figure
figure_layout = {
    'width': '800px',
    'height': '580px',
    'border': '3px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 2, center = (24.288292, 11.701573), layout=figure_layout)

# Create the heatmap layer for city locations and humidity
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=city_humidity.max(),
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
city_data_filtered = city_data.loc[(city_data['Max Temp'].between(70, 80)) & (city_data['Wind Speed'] < 5) & (city_data['Cloudiness'] < 5)].copy()  
city_data_clean = city_data_filtered.dropna()
city_data_clean

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,Les Cayes,18.20,-73.75,78.28,82,0,2.82,HT,1600397494
55,Yinchuan,38.47,106.27,71.73,22,0,2.42,CN,1600397512
267,Cayenne,4.93,-52.33,77.00,94,0,2.24,GF,1600397609
331,Piaçabuçu,-10.41,-36.43,71.55,83,0,3.49,BR,1600397645
409,Payson,40.04,-111.73,75.20,57,1,4.36,US,1600397684
424,Washington,47.50,-120.50,71.01,52,0,3.00,US,1600397168
499,Copperas Cove,31.12,-97.90,75.20,73,1,4.70,US,1600397726
519,Alenquer,-1.94,-54.74,78.80,94,0,4.00,BR,1600397731
549,Chaman,30.92,66.45,73.36,18,0,2.01,PK,1600397749
582,Seoul,37.57,126.98,77.00,41,1,3.36,KR,1600397587


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = city_data_clean
hotel_df = hotel_df.reset_index(drop=True)

In [39]:
hotel_df['Hotel Name'] = ""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create a dictionary variable to store the parameters
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
}

for index, row in hotel_df.iterrows():

    # get restaurant type from DataFrame
    lat_longs = f"{row['Lat']},{row['Lng']}"

    # add a new key to our params dictionary that will store our keyword to search for
    params['location'] = lat_longs

    # assemble url and make API request
#    print(f"Retrieving Results for Index {index}: {row['City']} {lat_longs}.")
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"The closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print(f"No hotel found in {row['City']}")

hotel_df

The closest hotel in Les Cayes is Villa Orphee.
The closest hotel in Yinchuan is JJ Inns - Yinchuan New Moon Square Branch.
The closest hotel in Cayenne is Hôtel Le Dronmi.
The closest hotel in Piaçabuçu is Piaçabuçu News.
The closest hotel in Payson is Quality Inn Payson I-15.
The closest hotel in Washington is Cashmere Cider Mill Guest Suites, Weddings & Tasting Room.
The closest hotel in Copperas Cove is Super 8 by Wyndham Copperas Cove.
The closest hotel in Alenquer is Detinha Hotel.
The closest hotel in Chaman is Niamatt parda Center.
The closest hotel in Seoul is LOTTE HOTEL SEOUL.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Les Cayes,18.20,-73.75,78.28,82,0,2.82,HT,1600397494,Villa Orphee
1,Yinchuan,38.47,106.27,71.73,22,0,2.42,CN,1600397512,JJ Inns - Yinchuan New Moon Square Branch
2,Cayenne,4.93,-52.33,77.00,94,0,2.24,GF,1600397609,Hôtel Le Dronmi
3,Piaçabuçu,-10.41,-36.43,71.55,83,0,3.49,BR,1600397645,Piaçabuçu News
4,Payson,40.04,-111.73,75.20,57,1,4.36,US,1600397684,Quality Inn Payson I-15
5,Washington,47.50,-120.50,71.01,52,0,3.00,US,1600397168,"Cashmere Cider Mill Guest Suites, Weddings & T..."
6,Copperas Cove,31.12,-97.90,75.20,73,1,4.70,US,1600397726,Super 8 by Wyndham Copperas Cove
7,Alenquer,-1.94,-54.74,78.80,94,0,4.00,BR,1600397731,Detinha Hotel
8,Chaman,30.92,66.45,73.36,18,0,2.01,PK,1600397749,Niamatt parda Center
9,Seoul,37.57,126.98,77.00,41,1,3.36,KR,1600397587,LOTTE HOTEL SEOUL


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the figure
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…